# Title

Description of your analysis

## Load packages

`hise`: The HISE SDK package for R  
`purrr`: Functional programming in R  
`furrr`: Parallel functional programming

In [8]:
library(dplyr)
library(hise)
library(purrr)
library(furrr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
plan("multicore", workers = 8)

## Helper functions

In [14]:
cache_uuid_path <- function(uuid) {
    if(!dir.exists(paste0("cache/", uuid))) {
        hise_res <- cacheFiles(list(uuid))
    }
    
    cache_path <- paste0("cache/",uuid)
    cache_file <- list.files(cache_path, full.names = TRUE)
    
    cache_file
}

## Identify files for use in HISE

In [5]:
search_id <- "polonium-tin-curium"

Retrieve the list of files stored in our HISE project store

In [9]:
ps_files <- listFilesInProjectStores(list("cohorts"))
ps_files <- map(
    ps_files$files, 
    function(l) {
        l <- l[c("id", "name")]
        as.data.frame(l)
    }) %>%
  list_rbind()

Filter for files from the previous notebook using our search_id and the .tar extension

In [10]:
tar_files <- ps_files %>%
  filter(grepl(search_id, name)) %>%
  filter(grepl(".tar$", name))

In [12]:
tar_files$name

[1] "polonium-tin-curium/diha_BR1_Female_Negative_h5ads_2024-05-05.tar"
[2] "polonium-tin-curium/diha_BR1_Female_Positive_h5ads_2024-05-05.tar"
[3] "polonium-tin-curium/diha_BR1_Male_Negative_h5ads_2024-05-05.tar"  
[4] "polonium-tin-curium/diha_BR1_Male_Positive_h5ads_2024-05-05.tar"  
[5] "polonium-tin-curium/diha_BR2_Female_Negative_h5ads_2024-05-05.tar"
[6] "polonium-tin-curium/diha_BR2_Female_Positive_h5ads_2024-05-05.tar"
[7] "polonium-tin-curium/diha_BR2_Male_Negative_h5ads_2024-05-05.tar"  
[8] "polonium-tin-curium/diha_BR2_Male_Positive_h5ads_2024-05-05.tar"

## Cache and unpack files

In [15]:
future_walk(
    tar_files$id,
    function(uuid) {
        tar_file <- cache_uuid_path(uuid)
        untar_call <- paste("tar -xf", tar_file)
        system(untar_call)
    }
)

In [17]:
sample_files <- list.files("sample_h5ad", full.names = TRUE)

In [18]:
head(sample_files)

[1] "sample_h5ad/PB00001-01_BR1001_Flu-Year-1-Day-0.h5ad"
[2] "sample_h5ad/PB00002-01_BR1002_Flu-Year-1-Day-0.h5ad"
[3] "sample_h5ad/PB00003-01_BR1003_Flu-Year-1-Day-0.h5ad"
[4] "sample_h5ad/PB00004-01_BR1004_Flu-Year-1-Day-0.h5ad"
[5] "sample_h5ad/PB00006-01_BR1005_Flu-Year-1-Day-0.h5ad"
[6] "sample_h5ad/PB00007-01_BR1003_Flu-Year-1-Day-7.h5ad"

## Perform analysis

## Write analysis outputs

In [ ]:
out_path <- "output"
if (!dir.exists(out_path)) {
    dir.create(out_path)
}

In [ ]:
out_file <- paste0(out_path, "/analysis_result_", Sys.Date(), ".csv")
write.csv(result, out_file)

## Upload results to HISE

In [ ]:
study_space_uuid <- ""
title <- paste0("CertPro Analysis Result ", Sys.Date())

In [ ]:
search_id <- ids::proquint(n_words = 3)
search_id

In [ ]:
in_files <- list(file_uuid)
in_files

In [ ]:
out_files <- list(out_file)
out_files

In [ ]:
uploadFiles(
    studySpaceId = study_space_uuid,
    title = title,
    files = out_files,
    inputFileIds = in_list,
    store = "project",
    destination = search_id
)

## Session Info

In [ ]:
import session_info
session_info.show()